In [2]:
import os
import re
import json
import pandas as pd
import numpy as np
from functools import partial
from IPython.core.debugger import set_trace
from tqdm import tqdm_notebook
#from multiprocessing import Pool, Process
from ckonlpy.tag import Twitter
import json
import ijson
from gensim.models import Word2Vec, FastText

In [3]:
with open('twitter_data/qry.json') as f:
    qry = json.load(f)
    
new_words = list(set(sum([re.findall(r'#([^ ]+)', v)+[k] for k,v in qry.items()], [])))

In [4]:
new_words

['eider',
 '몽벨',
 'yoshidaporter',
 'yojiyamamoto',
 'dailypaper',
 'jwanderson',
 '카피탈',
 'prada',
 'barrel',
 'dynafit',
 'ragandbone',
 'canadagoose',
 'abecrombiefitch',
 '몽클레르',
 '아디다스오리지널',
 'incase',
 'helmutlang',
 'pearlygates',
 '99퍼센트이즈',
 '걸스돈크라이',
 'undftd',
 'brixton',
 'saintlaurent',
 'cherryla',
 '스포츠웨어',
 'poloralphlauren',
 'stoneisland',
 'thursdayisland',
 'charms',
 'lafuma',
 '헤리티지플로스',
 '의류',
 'doublet',
 'bally',
 'gucci',
 '와이쓰리',
 'cutlerandgross',
 '미유미유',
 'pleatsplease',
 '니들스',
 'needles',
 'plac',
 'hagt',
 '이자벨마랑',
 'columbia',
 '폴로랄프로렌',
 '잔스포츠',
 '발망',
 '제이린드버그',
 '버켄스탁',
 '에이프',
 '미즈노',
 '버커루',
 '컬럼비아',
 'cpcompany',
 'abecrombie',
 '생로랑',
 'streetwear',
 'nerdy',
 'ferragamo',
 '뉴발란스',
 '인케이스',
 '닥터마틴',
 '레이닝챔프',
 '레디메이드',
 '젠틀몬스터',
 'yoshida',
 'readymade',
 '페레가모',
 '베자',
 'tods',
 '앰씨앰',
 '디스커버리',
 'salvatoreferregamo',
 '앨앰씨',
 'plasticisland',
 'soup',
 '비츠바이닥터드레',
 '디키즈',
 'vejasneakers',
 '허프',
 'series',
 '참스',
 '톰스슈즈',
 '옷',
 '스포츠패션',
 'mun

In [6]:
twitter = Twitter()
twitter.add_dictionary(new_words, 'Noun')

In [8]:
srcdir = 'twitter_data'
textcol = 'text'
fnames = [srcdir + '/' + fname for fname in os.listdir(srcdir)];fnames

['twitter_data/032c (2010.01.01-2019.01.31).pkl',
 'twitter_data/1017alyx9sm (2010.01.01-2019.01.31).pkl',
 'twitter_data/8seconds (2010.01.01-2019.01.31).pkl',
 'twitter_data/99percentis (2010.01.01-2019.01.31).pkl',
 'twitter_data/a.p.c (2010.01.01-2019.01.31).pkl',
 'twitter_data/aape (2010.01.01-2019.01.31).pkl',
 'twitter_data/abecrombiefitch (2010.01.01-2019.01.31).pkl',
 'twitter_data/acnestudios (2010.01.01-2019.01.31).pkl',
 'twitter_data/acoldwall (2010.01.01-2019.01.31).pkl',
 'twitter_data/acronym (2010.01.01-2019.01.31).pkl',
 'twitter_data/adererror (2010.01.01-2019.01.31).pkl',
 'twitter_data/adidas (2010.01.01-2019.01.31).pkl',
 'twitter_data/adidasoriginal (2010.01.01-2019.01.31).pkl',
 'twitter_data/advisoryboardcrystals (2010.01.01-2019.01.31).pkl',
 'twitter_data/aecawhite (2010.01.01-2019.01.31).pkl',
 'twitter_data/aimeleondore (2010.01.01-2019.01.31).pkl',
 'twitter_data/aldo (2010.01.01-2019.01.31).pkl',
 'twitter_data/alexandermcqueen (2010.01.01-2019.01.31).pk

In [9]:
tokens_dict = {}
for fname in tqdm_notebook(fnames):
    try:
        item, tokenized = tokenize(fname, textcol)
        tokens_dict[item] = tokenized
    except:
        pass

In [10]:
with open('twitter_data/tokens_dict.json', 'w') as f:
    json.dump(tokens_dict, f)

In [3]:
with open('twitter_data/tokens_dict.json') as f:
    tokens_dict = json.load(f)

In [61]:
w2v = Word2Vec(sum(tokens_dict.values(), []), size=100, window=2, min_count=10, workers=4, iter=100, sg=1)
w2v.save('twitter_data/word2vec.model')

In [62]:
w2v.init_sims(replace=True)

In [ ]:
w2v.load('twitter_data/word2vec.model')

In [81]:
w2v.wv.most_similar(positive=['아디다스오리지널'], topn=10)

NameError: name 'w2v' is not defined

In [4]:
sentences = iter(sum(tokens_dict.values(), []))
fasttext = FastText(sentences, size=100, window=2, min_count=100, workers=4, iter=5, sg=1)

MemoryError: 

In [4]:
sentences = sum(tokens_dict.values(), [])
fasttext = FastText(size=100, window=2, min_count=10)
fasttext.build_vocab(sentences=sentences)
fasttext.train(sentences=sentences, total_examples=len(sentences), epochs=10)

MemoryError: 

In [14]:
fasttext.wv.most_similar(positive=['심플'], topn=10)

TypeError: unsupported operand type(s) for ** or pow(): 'NoneType' and 'int'

In [8]:
class Tokens:
    def __init__(self, tokens_dict):
        self.tokens_dict = tokens_dict
        self.length = 0
        
    def __iter__(self):
        for _, v in self.tokens_dict.items():
            yield from v

    def __len__(self):
        if self.length == 0:
            for fname in self.fnames:
                self.length += len(pd.read_pickle(fname))
            
        return self.length

In [18]:
l = ijson.items(f, 'crocs.item')

In [71]:
sentences = Sentences(fnames, 'text')

In [72]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor(
    min_frequency=5,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)

word_extractor.train(sentences)
words = word_extractor.extract()

training was done. used memory 0.138 Gbry 0.137 Gb
all cohesion probabilities was computed. # words = 3517
all branching entropies was computed # words = 7926
all accessor variety was computed # words = 7926


In [116]:
iter_sent = iter(sentences)

In [127]:
next(iter_sent)

'iamhanbonnie 오빠에게 선물해준 두번째 신발  나도 하나 질러버려서  느낌으로 강남 aldo 커플신발'

In [128]:
preproc(pd.read_pickle('twitter_data/aldo (2010.01.01-2019.02.07).pkl').droplevel(0,axis=1).text.iloc[4])

{'urls': ['http://ift.tt/se4k81',
  'http://ift.tt/1q8ylqz',
  '..pic.twitter.com/u6btdf62t3'],
 'hashtags': ['강남', 'aldo', '커플신발'],
 'mention': [],
 'text': 'iamhanbonnie 오빠에게 선물해준 두번째 신발  나도 하나 질러버려서  느낌으로'}

In [129]:
pd.read_pickle('twitter_data/aldo (2010.01.01-2019.02.07).pkl').droplevel(0,axis=1).text.iloc[4]

'#강남 http://ift.tt/SE4k81\xa0 http://ift.tt/1q8Ylqz\xa0 iamhanbonnie 오빠에게 선물해준 두번째 신발 #ALDO 나도 하나 질러버려서 #커플신발 느낌으로..pic.twitter.com/u6BTDf62T3'

In [115]:
import math

def word_score(score):
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy)\n')
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True)[:100]:
    print('%s     (%d, %.3f, %.3f)' % (
            word, 
            score.leftside_frequency, 
            score.cohesion_forward,
            score.right_branching_entropy
            )
         )

단어   (빈도수, cohesion, branching entropy)

freitag     (355, 0.945, 3.552)
크록스     (880, 0.989, 3.152)
..     (30, 0.714, 3.280)
프라이탁     (244, 0.877, 3.000)
살로몬     (121, 0.943, 2.805)
알도     (165, 0.859, 2.801)
crocs     (601, 0.914, 2.736)
allsaints     (449, 0.941, 2.693)
...     (15, 0.598, 2.860)
!!     (7, 0.700, 2.690)
윤아     (780, 0.996, 2.206)
ㅋㅋ     (61, 0.910, 2.210)
올세인츠     (319, 0.953, 1.986)
korea     (26, 0.789, 2.160)
daily     (51, 0.799, 2.077)
헨리     (45, 1.000, 1.838)
신발     (140, 0.505, 2.500)
에서     (34, 0.425, 2.656)
^^     (15, 1.000, 1.714)
ㅎㅎ     (19, 1.000, 1.667)
aldo     (107, 0.527, 2.283)
행사     (43, 0.796, 1.855)
회사     (30, 1.000, 1.600)
yoona     (313, 0.957, 1.638)
샌들     (27, 0.818, 1.787)
re     (39, 0.443, 2.373)
너무     (41, 0.837, 1.721)
브랜드     (28, 0.907, 1.609)
컬렉션     (37, 0.887, 1.627)
salomon     (71, 0.736, 1.799)
ㅋㅋㅋ     (35, 0.723, 1.796)
함께     (49, 0.942, 1.517)
캠페인     (29, 0.874, 1.587)
같은     (19, 0.613, 1.934)
글로벌     (29, 0.983, 1.

In [7]:
def preproc(text, remove_url=True, remove_mention=True, remove_hashtag=False):
    LINEBREAK = r'\n' # str.replace에서는 r'\n'으로 검색이 안된다
    RT = ' rt '
    EMOJI = r'[\U00010000-\U0010ffff]'
    DOTS = '…'
    LONG_BLANK = r'[ ]+'
    
    # \u3131-\u3163\uac00-\ud7a3 는 한글을 의미함
    # URL = r'(?P<url>(https?://)?(www[.])?[^ \u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b([^ \u3131-\u3163\uac00-\ud7a3]*))'
    URL1 = r'(?:https?:\/\/)?(?:www[.])?[^ :\u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b(?:[^ \u3131-\u3163\uac00-\ud7a3]*)'
    URL2 = r'pic.twitter.com/[a-zA-Z0-9_]+'
    URL = '|'.join((URL1, URL2))
    
    HASHTAG = r'#(?P<inner_hashtag>[^ #@]+)'
    MENTION = r'@(?P<inner_mention>[^ #@]+)' 
    
    #PTNS = '|'.join((LINEBREAK, RT, URL, HASHTAG, MENTION, EMOJI))
    
    #out = {}
    text = re.sub('|'.join((LINEBREAK, RT, EMOJI, DOTS)), '', text.lower())
    
    if remove_url:
        text = re.sub(URL, '', text)

    if remove_mention:
        text = re.sub(MENTION, '', text)        
    else:
        text = re.sub(MENTION, ' \g<inner_mention>', text)
        
    if remove_hashtag:
        text = re.sub(HASHTAG, '', text)        
    else:
        text = re.sub(HASHTAG, ' \g<inner_hashtag>', text)
        
    return re.sub(LONG_BLANK, ' ', text).strip()


def tokenize(fname, textcol):
    df = pd.read_pickle(fname)
    item = df.columns.levels[0][0]
    tokenized = []
    
    for doc in df.droplevel(0, axis=1)[textcol]:
        text = preproc(doc)
        _tokenized = [t[0] for t in twitter.pos(text, norm=True, stem=True) if t[1] not in ['Punctuation', 'Josa']]
        tokenized.append(_tokenized)
    
    return item, tokenized

In [10]:
preproc(pd.read_pickle('twitter_data/allsaints (2010.01.01-2019.02.07).pkl').droplevel(0,axis=1).text.iloc[210], remove=True)

{'urls': ['http://www.fashionn.com/board/read_new.php?table=1006&number=21030&sns_number=989523…',
  'pic.twitter.com/2rys36ronu'],
 'hashtags': ['올세인츠', '올세인츠비치컬렉션pic.twitter.com/2rys36ronu'],
 'mention': [],
 'text': "올세인츠,  여름 한정판 '헌팅턴 비치 컬렉션' 출시"}

In [146]:
preproc(pd.read_pickle('twitter_data/allsaints (2010.01.01-2019.02.07).pkl').droplevel(0,axis=1).text.iloc[42])

'수퍼스타 커플신발 신세계백화점강남점 시댁선물사러나왔다가 allsaints 거울이좋네 ㅋㅋㅋ 얼마만에 같이나온건지~ 데이트하는기분'

In [145]:
pd.read_pickle('twitter_data/allsaints (2010.01.01-2019.02.07).pkl').droplevel(0,axis=1).text[42]

'#수퍼스타#커플신발#신세계백화점강남점 #시댁선물사러나왔다가 #ALLSAINTS #거울이좋네 ㅋㅋㅋ#얼마만에 같이나온건지~#데이트하는기분 https://www.instagram.com/p/-8DHERpvz-/\xa0'

In [147]:
def preproc_old(text, url=True, mention=True, hashtag=True, remove=True):
    LINEBREAK = r'\n' # str.replace에서는 r'\n'으로 검색이 안된다
    RT = ' rt '
    EMOJI = r'[\U00010000-\U0010ffff]'
    
    #URL = r'(?P<url>(https?://)?(www[.])?[^ \u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b([^ \u3131-\u3163\uac00-\ud7a3]*))'
    URL = r'(?:https?:\/\/)?(?:www[.])?[^ :\u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b(?:[^ \u3131-\u3163\uac00-\ud7a3]*)'
    # \u3131-\u3163\uac00-\ud7a3 는 한글을 의미함
    HASHTAG = r'(#[^ #@]+)'
    MENTION = r'(@[^ #@]+)' 
    
    PTNS = '|'.join((LINEBREAK, RT, URL, HASHTAG, MENTION, EMOJI))
    
    out = {}
    text = re.sub('|'.join((LINEBREAK,RT)), '', text.lower())
    
    if url:
        urls = re.findall(URL, text) #[m.groupdict()['url'] for m in re.finditer(URL, text)]
        urls = [url.replace('\xa0', '') for url in urls]
        out['urls'] = urls
        
    if hashtag:
        hashtags = re.findall(HASHTAG, text)
        out['hashtags'] = [h[1:] for h in hashtags]
        
    if mention:
        mentions = re.findall(MENTION, text)
        out['mention'] = [m[1:] for m in mentions]
        
    if remove:    
        text = re.sub(PTNS, '', text)
        
    out['text'] = text.strip()
    return out